In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

# Read Data

In [132]:
# GALAXY DATASET
def DatasetGALAXY():
    df = pd.read_csv('../../Data/galaxymorphology/dataset1_sydney.csv')
#    df.head()
#    df['target'].value_counts()
#     mapping = {
#         'elliptical': 0,
#         'spiral':1
#     }
#     df['target'] = df['target'].map(mapping)
    x = df.sample(df.shape[0])
    df = x.iloc[:, :5]
    df['target'] = x.iloc[:,-1]
    return df

df = DatasetGALAXY()
header = df.columns

In [47]:
#CAR DATASET
def DatasetCAR():
    df = pd.read_csv('../../Data/dataset1-car/car_dataset.csv')
    
    #     df['buying'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
    #     df['maintenance'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
    #     df['doors'].replace(['2', '3', '4', '5more'],[2,3,4,5], inplace = True)
    #     df['persons'].replace(['2', '4', 'more'],[2,4,5], inplace = True)
    #     df['lug_boot'].replace(['small', 'med', 'big'],[1,2,3], inplace = True)
    #     df['safety'].replace(['high', 'med', 'low'],[3,2,1], inplace = True)
    #     df['target'].replace(['unacc', 'acc', 'good', 'vgood'],[0,1,2,3], inplace = True)
    #     df= df[df.target !=3]
    #     df= df[df.target !=2]
    print(df['target'].nunique())
    df = df.sample(df.shape[0])
    return df

df = DatasetCAR()
header = df.columns

4


In [123]:
#IRIS DATASET
def DatasetIRIS():
    df = pd.read_csv('../../Data/iris/iris.data')
    df['target'].replace(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],[0,1,2], inplace = True)
    df = df.sample(df.shape[0])
    return df
df = DatasetIRIS()
header = df.columns

In [126]:
#TITANIC DATASET
def DatasetTITANIC():
    df = pd.read_csv('../../Data/Titanic/train.csv')
    df.drop(columns = [ 'Cabin','Embarked', 'Ticket','Name','PassengerId'], inplace=True)
    df['Sex'].replace(['male', 'female'],[0,1], inplace = True)
    df =df.dropna()
    
    return df
df = DatasetTITANIC()
header = df.columns

In [3]:
df

,u-g,g-r,r-i,i-z,ecc,target
457,1.58693,0.76100,0.40371,0.30242,0.902340,spiral
308,1.51834,0.64705,0.34067,0.24506,0.858611,spiral
447,1.90540,0.77048,0.38798,0.23775,0.818126,spiral
221,1.94276,0.87164,0.42491,0.30789,0.938287,elliptical
106,1.78496,0.77980,0.37753,0.28232,0.829166,elliptical
...,...,...,...,...,...,...
70,1.78528,0.94088,0.37692,0.36588,0.794977,elliptical
28,1.88772,0.85597,0.41393,0.32151,0.759583,elliptical
66,1.97871,1.02908,0.41819,0.34289,0.717513,elliptical
254,1.93122,1.06486,0.43941,0.36051,0.864887,elliptical


# Decision Trees

In [133]:
"""
We took help from the internet for the equations, and understanding how to apply some methods into coding format
"""

'\nWe took help from the internet for the equations, and understanding how to apply some methods into coding format\n'

In [166]:
#Counts the number of each type of example in a dataset
def count_targets(rows):
    count = {}  # a dictionary of label -> count.
    for row in rows:
        label = row[-1]
        if label not in count:
            count[label] = 0
        count[label] += 1
    return count

In [167]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [168]:
# the splitting question
class Question:

    def __init__(self, column, value):
        self.column = column #1
        self.value = value #3
    
    #Check if a row answers the question
    def check_answer(self, example): 
        value = example[self.column]
        if is_numeric(value):
            return value >= self.value
        else:
            return value == self.value
    
    # prints the question
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return f" {header[self.column]} {condition} {str(self.value)}?"


In [169]:
# splits the dataset according to the question
# true rows satisfy the question 
def split(rows, question):
    correct_rows, wrong_rows = [], []
    for row in rows:
        if question.check_answer(row):
            correct_rows.append(row)
        else:
            wrong_rows.append(row)
    return correct_rows, wrong_rows

In [170]:
#Calculate the Gini Impurity for a list of rows.
# gini equation was taken from here
#https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity

def gini_impurity(rows):
    count = count_targets(rows)
    impurity = 1
    for label in count:
        label_probability = count[label] / float(len(rows))
        impurity -= label_probability**2
    return impurity

In [171]:
# just like in the quiz
def info_gain(left, right, current_uncertainty):
    probability = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - probability * gini_impurity(left) - (1 - probability) * gini_impurity(right)

In [172]:
def best_splitting_feature(rows):
    
    highest_info_gain = 0  # keep track IG
    highest_info_gain_question = None 
    current_uncertainty = gini_impurity(rows)
    n_features = len(rows[0]) - 1 

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column
        
        for val in values:  # for each value

            question = Question(col, val)

            correct_rows, wrong_rows = split(rows, question)

            # Skip split if it doesn't divide the dataset.
            if len(correct_rows) == 0 or len(wrong_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(correct_rows, wrong_rows, current_uncertainty)

            if gain >= highest_info_gain:
                highest_info_gain, highest_info_gain_question = gain, question

    return highest_info_gain, highest_info_gain_question

In [173]:
class Leaf_node:
    def __init__(self, rows):
        self.predictions = count_targets(rows)


In [174]:
class Decision_Node:

    def __init__(self, question, right_branch, wrong_branch):
        self.question = question
        self.right_branch = right_branch
        self.wrong_branch = wrong_branch

In [181]:
# Tree building function
def build_tree_function(rows):

    info_gain, question = best_splitting_feature(rows)

    if info_gain == 0:
        return Leaf_node(rows)
    
    correct_rows, wrong_rows = split(rows, question)
    right_branch = build_tree_function(correct_rows)
    left_branch = build_tree_function(wrong_rows)

    return Decision_Node(question, right_branch, left_branch)


In [176]:
def classify(row, node):
    
    if isinstance(node, Leaf_node):
        return node.predictions

    # split
    if node.question.check_answer(row):
        return classify(row, node.right_branch)
    else:
        return classify(row, node.wrong_branch)

In [177]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
        
    return probs

In [178]:
def get_overall_cm(df):
    classes = df['target'].nunique()
    if classes < 2:
        classes = 2
    overall_cm = []
    for i in range(classes):
        overall_cm.append( [0] * classes)
    return overall_cm

In [179]:
def Kfold(df,n_folds, test_size, overall_cm):
    # Iterate through all folds
    predicted= []
    actual = []
    for i in range(n_folds):
        print("\n\n\n\nTHIS IS FOLD:", i)
        #split test and train
        test = df.iloc[test_size * i : test_size*i+test_size].to_numpy()
        train = df.iloc[test_size:].to_numpy()

        ####### THIS IS WHERE YOU CHANGE THE ALG #######
        #start training and testing the dataset
        my_tree = build_tree_function(train)
        
        
        for row in test:
            most_probable = print_leaf(classify(row, my_tree))
            predicted += [next(iter(most_probable))]
            actual += [row[-1]]
            print("Actual: %s. Predicted: %s" %(row[-1], print_leaf(classify(row, my_tree))))
        # generate and append cm
        overall_cm = overall_cm + confusion_matrix(actual, predicted)
    print(classification_report(actual, predicted ))
    return overall_cm


In [182]:
if __name__ == "__main__":
    # KFOLD take in n folds and the size of dataset
    
    # DATASET1 SYDNEY
    n_folds = 2
    test_size = int(df.shape[0]/n_folds)

    # get overall cm returns a Confusion matrix with 0's shape depends on (classes)
    dtcm = Kfold(df,n_folds, test_size, get_overall_cm(df))
    print(f"\n\n\n Confusion Matrix\n{dtcm}")





THIS IS FOLD: 0
Actual: spiral. Predicted: {'spiral': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: spiral. Predicted: {'spiral': '100%'}
Actual: spiral. Predicted: {'spiral': '100%'}
Actual: spiral. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: spiral. Predicted: {'spiral': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: spiral. Predicted: {'spiral': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'elliptical': '100%'}
Actual: elliptical. Predicted: {'spiral': '100%'}
Ac